### Feature Selection

In [2]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
from tableone import TableOne

Import Data

In [3]:
POST_METADATA = pd.read_csv('~/Desktop/POST_METADATA.csv').set_index('External ID')

In [36]:
POST_METADATA.head()

,data_type,IntervalName,Education Level,Occupation,"Soft drinks, tea or coffee with sugar (corn syrup, maple syrup, cane sugar, etc)","Diet soft drinks, tea or coffee with sugar (Stevia, Equal, Splenda etc)","Fruit juice (orange, apple, cranberry, prune etc.)",Water,"Alcohol (beer, brandy, spirits, hard liquor, wine, aperitif, etc.)","Yogurt or other foods containing active bacterial cultures (kefir, sauerkraut)",...,consent_age,fecalcal,fecalcal_ng_ml,interval_days,s__Bacteroides_uniformis,s__Faecalibacterium_prausnitzii,s__Parabacteroides_distasonis,visit_num,week_num,diagnosis
External ID,,,,,,,,,,,,,,,,,,,,,
CSM5FZ4MM,metatranscriptomics,Stool Collection #6,Master's degree,Paid,"Yesterday, 1 to 2 times","Yesterday, 1 to 2 times","No, I did not consume these products in the la...","Yesterday, 3 or more times","Yesterday, 1 to 2 times","No, I did not consume these products in the la...",...,37.0,240.090243,19.888863,25.856009,0.979500,0.601848,1.022414,20.409917,41.0,UC
CSM5FZ4MM1,metagenomics,Stool Collection #6,Master's degree,Paid,"Yesterday, 1 to 2 times","Yesterday, 1 to 2 times","No, I did not consume these products in the la...","Yesterday, 3 or more times","Yesterday, 1 to 2 times",Unknown,...,58.0,223.285885,317.063094,26.000000,0.979500,0.601848,1.022414,25.385270,48.0,UC
CSM5MCUOM,metatranscriptomics,Stool Collection #6,Professional/Doctoral degree,Paid,"No, I did not consume these products in the la...","No, I did not consume these products in the la...",Unknown,"Yesterday, 3 or more times",Within the past 2 to 3 days,"Yesterday, 1 to 2 times",...,67.0,222.001161,68.541729,88.000000,1.316983,0.389485,0.736020,9.000000,15.0,UC
CSM5MCUOM1,metagenomics,Stool Collection #6,Professional/Doctoral degree,Paid,"No, I did not consume these products in the la...","No, I did not consume these products in the la...",Within the past 4 to 7 days,"Yesterday, 3 or more times",Within the past 2 to 3 days,"Yesterday, 1 to 2 times",...,18.0,343.835485,175.609929,76.000000,1.316983,0.389485,0.916576,15.944661,31.0,UC
CSM5MCWQM,metagenomics,Stool Collection #9,Master's degree,Unknown,"Yesterday, 1 to 2 times","Yesterday, 1 to 2 times",Within the past 2 to 3 days,"Yesterday, 3 or more times",Within the past 2 to 3 days,"No, I did not consume these products in the la...",...,43.0,194.991146,248.368993,1.000000,0.715550,0.638939,0.171210,3.000000,17.0,UC


 ### Table division:

  - Target variable
  - Continuous variables
  - Categorical variables

In [5]:
Target = POST_METADATA.iloc[:,-1]
IBD_X_N = POST_METADATA.select_dtypes(include = ['float64', 'int64'])
IBD_X_C = POST_METADATA.loc[:,POST_METADATA.columns != 'diagnosis']
IBD_X_C = IBD_X_C.select_dtypes(include = [object])

Create a dataframe to summarize the results

In [6]:
FearSel = pd.DataFrame({'Variable': IBD_X_N.columns}).set_index('Variable')

In [7]:
X = IBD_X_N.copy()
y = Target

To compute univariate, use Tabla1

In [8]:
#upload data
data = POST_METADATA

# columns to summarize
columns = list(POST_METADATA.columns)

# columns containing categorical variables
categorical = list(IBD_X_C.columns)

# non-normal variables
nonnormal = list(IBD_X_N.columns)

# optionally, a categorical variable for stratification
groupby = ['diagnosis']

In [9]:
table1 = TableOne(data, columns,categorical, groupby, nonnormal, pval = True,
                  htest_name=True)
Univariable_table = table1.tableone
Univariable_table.columns = Univariable_table.columns.droplevel(0)
Univariable_table.reset_index(inplace=True)
Univariable_table.head()

,level_0,level_1,Missing,Overall,CD,UC,nonIBD,P-Value,Test
0,n,,,1064,496,296,272,,
1,"data_type, n (%)",metagenomics,0,531 (49.9),250 (50.4),145 (49.0),136 (50.0),0.928,Chi-squared
2,"data_type, n (%)",metatranscriptomics,,533 (50.1),246 (49.6),151 (51.0),136 (50.0),,
3,"IntervalName, n (%)",Stool Collection #1,0,12 (1.1),2 (0.4),4 (1.4),6 (2.2),0.005,Chi-squared (warning: expected count < 5)
4,"IntervalName, n (%)",Stool Collection #10,,27 (2.5),13 (2.6),10 (3.4),4 (1.5),,


Choosing significant features

In [10]:
Univariable_table['P-Value'] = pd.to_numeric(Univariable_table['P-Value'].str.replace("<0.001", "0", regex=True))
Univariable_table.head()

,level_0,level_1,Missing,Overall,CD,UC,nonIBD,P-Value,Test
0,n,,,1064,496,296,272,NaN,
1,"data_type, n (%)",metagenomics,0,531 (49.9),250 (50.4),145 (49.0),136 (50.0),0.928,Chi-squared
2,"data_type, n (%)",metatranscriptomics,,533 (50.1),246 (49.6),151 (51.0),136 (50.0),NaN,
3,"IntervalName, n (%)",Stool Collection #1,0,12 (1.1),2 (0.4),4 (1.4),6 (2.2),0.005,Chi-squared (warning: expected count < 5)
4,"IntervalName, n (%)",Stool Collection #10,,27 (2.5),13 (2.6),10 (3.4),4 (1.5),NaN,


In [11]:
Univariable_table = Univariable_table[Univariable_table['P-Value'] <= 0.05]
Univariable_table['Univariable'] = 1
Univariable_table_categorical = Univariable_table.iloc[0:-13,:]
Univariable_table_numerical = Univariable_table.iloc[-13:len(Univariable_table),:]
Univariable_table_categorical

/var/folders/qz/fy7nnm954f76spkqc86w7g_c0000gn/T/ipykernel_16085/2679104909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Univariable_table['Univariable'] = 1


,level_0,level_1,Missing,Overall,CD,UC,nonIBD,P-Value,Test,Univariable
3,"IntervalName, n (%)",Stool Collection #1,0,12 (1.1),2 (0.4),4 (1.4),6 (2.2),0.005,Chi-squared (warning: expected count < 5),1
28,"Education Level, n (%)",7th grade or less,0,212 (19.9),102 (20.6),38 (12.8),72 (26.5),0.000,Chi-squared,1
37,"Occupation, n (%)",Paid,0,346 (32.5),173 (34.9),75 (25.3),98 (36.0),0.000,Chi-squared (warning: expected count < 5),1
43,"Soft drinks, tea or coffee with sugar (corn sy...","No, I did not consume these products in the la...",0,243 (22.8),74 (14.9),100 (33.8),69 (25.4),0.000,Chi-squared,1
49,"Diet soft drinks, tea or coffee with sugar (St...","No, I did not consume these products in the la...",0,662 (62.2),336 (67.7),183 (61.8),143 (52.6),0.000,Chi-squared,1
55,"Fruit juice (orange, apple, cranberry, prune e...","No, I did not consume these products in the la...",0,291 (27.3),106 (21.4),71 (24.0),114 (41.9),0.000,Chi-squared,1
61,"Water, n (%)",Unknown,0,93 (8.7),48 (9.7),32 (10.8),13 (4.8),0.000,Chi-squared (warning: expected count < 5),1
67,"Alcohol (beer, brandy, spirits, hard liquor, w...","No, I did not consume these products in the la...",0,559 (52.5),295 (59.5),135 (45.6),129 (47.4),0.000,Chi-squared,1
73,Yogurt or other foods containing active bacter...,"No, I did not consume these products in the la...",0,454 (42.7),230 (46.4),109 (36.8),115 (42.3),0.000,Chi-squared,1
78,"Dairy (milk, cream, ice cream, cheese, cream c...","No, I did not consume these products in the la...",0,36 (3.4),21 (4.2),13 (4.4),2 (0.7),0.000,Chi-squared,1


In [12]:
Univariable_table_categorical[['Feature','By_Freq']] = Univariable_table_categorical.level_0.str.rsplit(',',1, expand= True)
Univariable_table_numerical[['Feature','By_Freq']] = Univariable_table_numerical.level_0.str.split(',',1, expand= True)

/var/folders/qz/fy7nnm954f76spkqc86w7g_c0000gn/T/ipykernel_16085/2014247253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Univariable_table_categorical[['Feature','By_Freq']] = Univariable_table_categorical.level_0.str.rsplit(',',1, expand= True)
/var/folders/qz/fy7nnm954f76spkqc86w7g_c0000gn/T/ipykernel_16085/2014247253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Univariable_table_categorical[['Feature','By_Freq']] = Univariable_table_categorical.level_0.str.rsplit(',',1, expand= True)
/var/f

In [13]:
Univariable_table_categorical = Univariable_table_categorical.drop(['level_0'],axis=1).set_index('Feature')
Univariable_table_numerical = Univariable_table_numerical.drop(['level_0','level_1'],axis=1).set_index('Feature')

In [14]:
List_categorical_selected = Univariable_table_categorical.index.tolist()
print(f'Columns number to keep: {len(List_categorical_selected)}')

Columns number to keep: 43


Find the distinct values for each characteristic and convert the data to a binary one-hot encoding

In [15]:
Keep_Categorical = IBD_X_C[IBD_X_C.columns[IBD_X_C.columns.isin(List_categorical_selected)]]
OneHot_table_categorical = pd.get_dummies(Keep_Categorical)
OneHot_table_categorical

,IntervalName_Stool Collection #1,IntervalName_Stool Collection #10,IntervalName_Stool Collection #11,IntervalName_Stool Collection #12,IntervalName_Stool Collection #13,IntervalName_Stool Collection #14,IntervalName_Stool Collection #15,IntervalName_Stool Collection #16,IntervalName_Stool Collection #17,IntervalName_Stool Collection #18,...,Bowel frequency during the day_06-Apr,Bowel frequency during the day_09-Jul,Bowel frequency during the day_Unknown,sex_Female,sex_Male,sex_Unknown,smoking status_Current smoker,smoking status_Former smoker,smoking status_Never smoked,smoking status_Unknown
External ID,,,,,,,,,,,,,,,,,,,,,
CSM5FZ4MM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
CSM5FZ4MM1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
CSM5MCUOM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
CSM5MCUOM1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
CSM5MCWQM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PSMB4MBSM1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
PSMB4MC5M,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
PSMB4MC5M1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


Set missing values to zero

In [16]:
FearSel['Univariable'] = Univariable_table_numerical['Univariable'].astype('int64')
FearSel['Univariable'] = FearSel['Univariable'].fillna(0)
FearSel['Univariable']

Variable
BMI                                1.0
CRP (mg/L)                         1.0
ESR (mm/hr)                        1.0
Height                             1.0
IntervalSequence                   0.0
Total Reads                        1.0
Weight                             1.0
Weight_n                           1.0
consent_age                        1.0
fecalcal                           1.0
fecalcal_ng_ml                     1.0
interval_days                      1.0
s__Bacteroides_uniformis           0.0
s__Faecalibacterium_prausnitzii    1.0
s__Parabacteroides_distasonis      1.0
visit_num                          0.0
week_num                           0.0
Name: Univariable, dtype: float64

Find important features with Random Forest model

In [17]:
rfmod = RandomForestClassifier().fit(X, y)

In [18]:
model = SelectFromModel(rfmod, prefit=True)

In [19]:
FearSel['RandomForest'] = model.get_support().astype('int64')
FearSel

,Univariable,RandomForest
Variable,,
BMI,1.0,0
CRP (mg/L),1.0,1
ESR (mm/hr),1.0,1
Height,1.0,1
IntervalSequence,0.0,0
Total Reads,1.0,0
Weight,1.0,0
Weight_n,1.0,1
consent_age,1.0,0


Find important features with Gradient Boosting Classifier model

In [20]:
gbmod = GradientBoostingClassifier().fit(X, y)

In [21]:
model = SelectFromModel(gbmod, prefit=True)

In [22]:
FearSel['GradientBoost'] = model.get_support().astype('int64')
FearSel

,Univariable,RandomForest,GradientBoost
Variable,,,
BMI,1.0,0,0
CRP (mg/L),1.0,1,1
ESR (mm/hr),1.0,1,1
Height,1.0,1,0
IntervalSequence,0.0,0,0
Total Reads,1.0,0,0
Weight,1.0,0,0
Weight_n,1.0,1,0
consent_age,1.0,0,0


Find important features with Support Vector Classifier model

In [23]:
svmmod = LinearSVC(C=0.01, penalty="l1",dual=False).fit(X, y)

In [24]:
model = SelectFromModel(svmmod, prefit=True)

In [25]:
FearSel['SVC'] = model.get_support().astype('int64')
FearSel

,Univariable,RandomForest,GradientBoost,SVC
Variable,,,,
BMI,1.0,0,0,1
CRP (mg/L),1.0,1,1,1
ESR (mm/hr),1.0,1,1,1
Height,1.0,1,0,1
IntervalSequence,0.0,0,0,1
Total Reads,1.0,0,0,0
Weight,1.0,0,0,1
Weight_n,1.0,1,0,1
consent_age,1.0,0,0,1


Find important features with Logistic Regression model

In [26]:
Log_R = LogisticRegression().fit(X, y)

In [27]:
model = SelectFromModel(Log_R, prefit=True)

In [28]:
FearSel['LogisticRegression'] = model.get_support().astype('int64')
FearSel

,Univariable,RandomForest,GradientBoost,SVC,LogisticRegression
Variable,,,,,
BMI,1.0,0,0,1,0
CRP (mg/L),1.0,1,1,1,0
ESR (mm/hr),1.0,1,1,1,0
Height,1.0,1,0,1,0
IntervalSequence,0.0,0,0,1,0
Total Reads,1.0,0,0,0,1
Weight,1.0,0,0,1,0
Weight_n,1.0,1,0,1,0
consent_age,1.0,0,0,1,0


All models summary

In [29]:
FearSel['Sum'] =  np.sum(FearSel,axis=1)
FearSel

,Univariable,RandomForest,GradientBoost,SVC,LogisticRegression,Sum
Variable,,,,,,
BMI,1.0,0,0,1,0,2.0
CRP (mg/L),1.0,1,1,1,0,4.0
ESR (mm/hr),1.0,1,1,1,0,4.0
Height,1.0,1,0,1,0,3.0
IntervalSequence,0.0,0,0,1,0,1.0
Total Reads,1.0,0,0,0,1,2.0
Weight,1.0,0,0,1,0,2.0
Weight_n,1.0,1,0,1,0,3.0
consent_age,1.0,0,0,1,0,2.0


The selection features threshold was set to a high value of 2.

In [30]:
FearSel = FearSel[FearSel['Sum'] >= 2]
FearSel

,Univariable,RandomForest,GradientBoost,SVC,LogisticRegression,Sum
Variable,,,,,,
BMI,1.0,0,0,1,0,2.0
CRP (mg/L),1.0,1,1,1,0,4.0
ESR (mm/hr),1.0,1,1,1,0,4.0
Height,1.0,1,0,1,0,3.0
Total Reads,1.0,0,0,0,1,2.0
Weight,1.0,0,0,1,0,2.0
Weight_n,1.0,1,0,1,0,3.0
consent_age,1.0,0,0,1,0,2.0
fecalcal,1.0,0,0,1,0,2.0


In [31]:
List_numerical_selected = FearSel.index.tolist()
print(f'column number to keep: {len(List_numerical_selected)}')

column number to keep: 11


Only the variables selected by the models are saved

In [32]:
Keep_numeric = IBD_X_N[IBD_X_N.columns[IBD_X_N.columns.isin(List_numerical_selected)]]
Keep_numeric

,BMI,CRP (mg/L),ESR (mm/hr),Height,Total Reads,Weight,Weight_n,consent_age,fecalcal,fecalcal_ng_ml,interval_days
External ID,,,,,,,,,,,
CSM5FZ4MM,16.945470,9.597698,98.184831,192.000000,5.524543e+07,112.0,24.000000,37.000000,240.090243,19.888863,25.856009
CSM5FZ4MM1,16.456143,40.320708,20.960983,152.000000,3.786562e+07,70.0,53.000000,58.000000,223.285885,317.063094,26.000000
CSM5MCUOM,15.711854,48.569000,36.625684,144.313066,2.553641e+05,64.0,116.000000,67.000000,222.001161,68.541729,88.000000
CSM5MCUOM1,31.439759,11.180201,12.256574,143.666757,6.052792e+06,86.0,98.000000,18.000000,343.835485,175.609929,76.000000
CSM5MCWQM,39.813988,54.326659,28.496137,164.418809,1.043272e+07,103.0,28.000000,43.000000,194.991146,248.368993,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
PSMB4MBSM1,25.104382,0.840428,44.057621,169.000000,5.611575e+07,92.0,77.528729,32.000000,194.538209,59.287025,50.000000
PSMB4MC5M,16.327456,13.047489,91.892652,167.000000,5.457551e+07,19.0,31.000000,25.000000,420.326372,138.643638,60.000000
PSMB4MC5M1,36.369043,9.849275,45.065309,118.000000,1.777290e+07,77.0,94.000000,49.000000,153.919317,391.352492,29.000000


Combining with the target variable

In [33]:
Feat_prediction = pd.concat([Keep_numeric,OneHot_table_categorical,Target],axis=1)
Feat_prediction

,BMI,CRP (mg/L),ESR (mm/hr),Height,Total Reads,Weight,Weight_n,consent_age,fecalcal,fecalcal_ng_ml,...,Bowel frequency during the day_09-Jul,Bowel frequency during the day_Unknown,sex_Female,sex_Male,sex_Unknown,smoking status_Current smoker,smoking status_Former smoker,smoking status_Never smoked,smoking status_Unknown,diagnosis
External ID,,,,,,,,,,,,,,,,,,,,,
CSM5FZ4MM,16.945470,9.597698,98.184831,192.000000,5.524543e+07,112.0,24.000000,37.000000,240.090243,19.888863,...,0,0,1,0,0,0,0,1,0,UC
CSM5FZ4MM1,16.456143,40.320708,20.960983,152.000000,3.786562e+07,70.0,53.000000,58.000000,223.285885,317.063094,...,0,0,1,0,0,0,0,0,1,UC
CSM5MCUOM,15.711854,48.569000,36.625684,144.313066,2.553641e+05,64.0,116.000000,67.000000,222.001161,68.541729,...,0,0,0,1,0,0,0,0,1,UC
CSM5MCUOM1,31.439759,11.180201,12.256574,143.666757,6.052792e+06,86.0,98.000000,18.000000,343.835485,175.609929,...,0,0,0,1,0,0,0,0,1,UC
CSM5MCWQM,39.813988,54.326659,28.496137,164.418809,1.043272e+07,103.0,28.000000,43.000000,194.991146,248.368993,...,0,0,0,0,1,0,0,0,1,UC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PSMB4MBSM1,25.104382,0.840428,44.057621,169.000000,5.611575e+07,92.0,77.528729,32.000000,194.538209,59.287025,...,0,0,0,1,0,0,0,1,0,CD
PSMB4MC5M,16.327456,13.047489,91.892652,167.000000,5.457551e+07,19.0,31.000000,25.000000,420.326372,138.643638,...,0,1,1,0,0,0,0,1,0,UC
PSMB4MC5M1,36.369043,9.849275,45.065309,118.000000,1.777290e+07,77.0,94.000000,49.000000,153.919317,391.352492,...,0,0,0,0,1,0,0,1,0,UC


Export data

In [34]:
Feat_prediction.to_csv('~/Desktop/Feat_prediction.csv')